# Segmenting and Clustering Neighborhoods in Toronto

### Scrape information from the wikipedia page on Toronto neighborhoods

- use BeautifulSoup for webscrapping the table

import libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

website for scrapping:

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

getting the entire website as lxml:

In [4]:
html_content = requests.get(url)

get text from the website

In [5]:
content = html_content.text
soup = BeautifulSoup(content, 'lxml')

find table:

In [6]:
# find table on the website
table = soup.find("table", attrs = {"class": "wikitable sortable"})

get table headers:

In [7]:
t_headers = []
for th in table.find_all("th"):
    t_headers.append(th.text.replace("\n",' ').strip())

t_headers

['Postal Code', 'Borough', 'Neighbourhood']

get data from table into a dictionary:

In [8]:
table_data = []
for tr in table.tbody.find_all("tr"): # each row in tbody of table is tr
    t_row = {}
    for td, th in zip(tr.find_all("td"), t_headers): # each cell in row is td
        t_row[th] = td.text.replace('\n','').strip()
    table_data.append(t_row)

### Wrangle the data

In [9]:
table_data = table_data[1:]  # remove first row (it is empty)

Convert dictionary into a dataframe:

In [10]:
df_table = pd.DataFrame(table_data)
df_table.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
df_table.shape

(180, 3)

Remove rows where Borough is Not assigned:

In [12]:
df_table = df_table[df_table['Borough']!='Not assigned']
df_table.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
df_table.shape

(103, 3)

Combine rows with same Postal Code/Borough:

In [14]:
df_table = pd.DataFrame(df_table.groupby(['Postal Code','Borough'], sort=False)['Neighbourhood'].apply(', '.join))
df_table.reset_index(inplace=True)
df_table.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
df_table.shape

(103, 3)

\* seems that there was no repeating Postal Code

For the rows with Not assigned Neighbourhood, apply Borough as the Neighbourhood name:

In [16]:
df_table.loc[df_table['Neighbourhood']=='Not assigned','Neighbourhood'] = df_table.loc[df_table['Neighbourhood']=='Not assigned','Borough']

Number of rows obtained after wrangling of the data:

In [17]:
print('There is {} rows in the dataframe'.format(df_table.shape[0]))

There is 103 rows in the dataframe


## Get spacial coordinates

- obtain spatial coordinates for each neighbourhood

In [18]:
import geocoder

loop through all the rows and get coordinates based on post code:

In [ ]:
for idx, address in enumerate(df_table['Postal Code']):
    address_full = address + ', Toronto, Ontario, Canada'
    
    coord = None
    i = 0
    while coord == None or i==20:
        coord = geocoder.arcgis(address_full).latlng
        i+=1
    df_table.loc[idx, 'Latitude'] = coord[0]
    df_table.loc[idx, 'Longitude'] = coord[1]

In [ ]:
df_table.head()

### Explore and cluster the neighborhoods in Toronto

Filter the neighbourhoods to get only those located in Toronto city

In [ ]:
df_toronto = df_table[df_table['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

In [ ]:
df_toronto.shape

Foursquare creditentials

In [ ]:
import json
with open("C:\\Users\\Pawel\\Documents\\Github\\credits_fs.json", 'r') as f:
    creds = json.load(f)
CLIENT_ID = creds['CLIENT_ID'] # your Foursquare ID
CLIENT_SECRET = creds['CLIENT_SECRET'] # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Get top 100 venues located within 500m using foursquare's API

In [ ]:
LIMIT = 100
RADIUS = 200

toronto_venues = []

for i in range(df_toronto.shape[0]):
    # url request to the foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit{}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        df_toronto.loc[i,'Latitude'], 
        df_toronto.loc[i,'Longitude'], 
        RADIUS, 
        LIMIT)
    
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    toronto_venues.append([(
        df_toronto.loc[i,'Postal Code'],
        df_toronto.loc[i,'Borough'],
        df_toronto.loc[i,'Neighbourhood'],
        df_toronto.loc[i,'Latitude'],
        df_toronto.loc[i,'Longitude'], 
        result['venue']['categories'][0]['name']) for result in results])

Convert to dataframe

In [ ]:
df_toronto_venues = pd.DataFrame(data=[item for venue_list in toronto_venues for item in venue_list],
                                columns=['Postal Code', 'Borough', 'Neighbourhood','Latitude','Longitude','Venue'])
df_toronto_venues.head()

In [ ]:
df_toronto_venues['Neighbourhood'].value_counts().shape

perform one-hot-encoding

In [ ]:
# one-hot-encoding for the venues
toronto_onehot = pd.get_dummies(df_toronto_venues[['Venue']], prefix="", prefix_sep="")

# move neighbourhood column to the front
toronto_onehot.insert(0, 'Neighbourhood', df_toronto_venues['Neighbourhood'])

# sum up venue types for each neighbourhood
toronto_group = toronto_onehot.groupby(['Neighbourhood']).mean().reset_index()

toronto_group.head()

#### Cluster neighbourhoods based on number of venue types

we use k-means to group neighbourhoods in 4 groups

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# get features based on which groups will be made
features = toronto_group.drop(['Neighbourhood'], axis=1)

# create k-means model and fit our data
kclusters = 3
kmeans = KMeans(n_clusters=kclusters, init='k-means++', n_init=20, max_iter=500, random_state=0).fit(features)

create new data frame with assigned cluster and top 3 venue types for each neighbourhood

In [ ]:
columns=['Cluster', 'Neighbourhood', 'top_1_venue', 'top_2_venue', 'top_3_venue']
toronto_clustered = pd.DataFrame(columns=columns)
for i in range(features.shape[0]):
    toronto_clustered.loc[i,columns[0]] = kmeans.labels_[i]
    toronto_clustered.loc[i,columns[1]] = toronto_group.loc[i, 'Neighbourhood']
    toronto_clustered.loc[i,columns[2:5]] = features.iloc[i,:].sort_values(ascending=False).index[0:3]

In [ ]:
toronto_clustered.head()

add borough and location data

In [ ]:
toronto = df_toronto[['Borough','Neighbourhood','Latitude','Longitude']]
toronto = toronto.join(toronto_clustered.set_index('Neighbourhood'), on='Neighbourhood', how='right')
toronto.head()

group by clusters

In [ ]:
# move cluster column to front
fixed_columns = [toronto.columns[4]] + list(toronto.columns[0:4]) + list(toronto.columns[5:])
toronto_cluster = toronto[fixed_columns]

# group by cluster
toronto_cluster.sort_values(by='Cluster', inplace=True)
toronto_cluster.reset_index(drop=True, inplace=True)
toronto_cluster.head()

How were the neighbourhoods classified:

In [ ]:
toronto_cluster['Cluster'].value_counts()

What was the main factor for each cluster:

In [ ]:
cluster0 = pd.melt(toronto_cluster[toronto_cluster['Cluster']==0][['top_1_venue','top_2_venue','top_3_venue']])['value'].value_counts()
cluster0[0:3]

In [ ]:
cluster1 = pd.melt(toronto_cluster[toronto_cluster['Cluster']==1][['top_1_venue','top_2_venue','top_3_venue']])['value'].value_counts()
cluster1[0:3]

In [ ]:
cluster2 = pd.melt(toronto_cluster[toronto_cluster['Cluster']==2][['top_1_venue','top_2_venue','top_3_venue']])['value'].value_counts()
cluster2[0:3]

\* it seems that:<br>
    - first cluster contains neighbourhoods where parks are most common<br>
    - second cluster contains neighbourhoods where cafe's are most common<br>
    - and third cluster contains neighbourhoods with no cafes or parks

#### Create map showing different clusters

In [ ]:
import folium

In [ ]:
toronto_map = folium.Map([toronto_cluster['Latitude'].median(), toronto_cluster['Longitude'].median()], zoom_start=12)

colors = ['red', 'blue', 'green', 'magenta']

for i in range(toronto_cluster.shape[0]):
        folium.CircleMarker(
        [toronto_cluster.loc[i, 'Latitude'], toronto_cluster.loc[i, 'Longitude']],
        color = colors[int(toronto_cluster.loc[i, 'Cluster'])],
        radius = 5,
        fill = True,
        fill_color = colors[int(toronto_cluster.loc[i, 'Cluster'])],
        fill_opacity = 0.7,
        popup = toronto_cluster.loc[i, 'Neighbourhood']
    ).add_to(toronto_map)

toronto_map